## 라이브러리 불러오기

In [1]:
# 라이브러리 import
import pandas as pd
import time
import datetime
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

## 크롬으로 리뷰 창 열기

In [2]:
# 크롬 옵션 설정
chrome_options=webdriver.ChromeOptions()
chrome_options.add_argument('--incognito') #시크릿모드의 브라우저 실행
chrome_options.add_argument('--no-sandbox') 
chrome_options.add_argument('--disable-setuid-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])

In [3]:
# 제품 창 열기(url 입력)
driver=webdriver.Chrome(options=chrome_options)
driver.get('https://www.amazon.com/OMEN-HP-Display-i9-12900H-Graphics/dp/B0CJM1XDVZ/ref=sr_1_14?crid=2W1BXJ7DBTOF3&keywords=hp%2Bomen&qid=1700742518&sprefix=hp%2Bomen%2Caps%2C330&sr=8-14&th=1')

***다음 셀을 실행하기 전에 크롬 창에서 로봇 방지 문자를 입력해야 하는지 확인해주세요.***

In [4]:
# '리뷰 더 보기' 버튼 클릭
option_button = driver.find_element(By.XPATH,'//*[@id="reviews-medley-footer"]/div[2]/a')
option_button.click()
time.sleep(1.5)

## 사용할 함수 정의

In [5]:
# 사용자 id 가져오기
def get_customer_id(i):
    try: # 미국에서 작성된 리뷰
        cu1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/div[1]/a')
    except: # 그 외의 국가에서 작성된 리뷰
        cu1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/div[1]/div')
    
    cu2 = cu1.text
    
    return cu2

# 별점 가져오기
def get_rating(i):
    try: # 미국에서 작성된 리뷰
        r1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/div[2]/a/i/span')
    except: # 그 외의 국가에서 작성된 리뷰
        r1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/div[2]/i/span')
    
    r2 = float(r1.get_attribute('innerText')[-3:])
    
    return r2

# 리뷰 제목 가져오기
def get_title(i):
    try: # 미국에서 작성된 리뷰
        t1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/div[2]/a/span[2]')
    except: # 그 외의 국가에서 작성된 리뷰
        t1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/div[2]/span[2]')
            
    t2 = t1.text
    
    return t2

# 리뷰가 작성된 국가와 날짜 가져오기
def get_country_date(i):
    
    d1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/span')
    d2 = d1.text

    ## country
    country_index = d2.find('에서')
    d3_country = d2[:country_index]

    ## date
    year_index = d2.find('년 ')
    month_index = d2.find('월 ')
    day_index = d2.find('일에 ')
    year = int(d2[year_index-4:year_index])
    month = int(d2[month_index-2:month_index])
    day = int(d2[day_index-2:day_index])

    d3_date = datetime.date(year, month, day).strftime("%Y-%m-%d")

    return d3_country, d3_date

# 리뷰 내용 가져오기 
def get_content(i):
    co1 = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]/div/div/div[4]/span')
    co2 = co1.text
    return co2

# 리뷰가 유용하다고 평가한 사람 수 가져오기
def get_helpful():
    helpful = []
    h_list = driver.find_elements(By.CLASS_NAME, 'cr-vote')
    for h in h_list:
        h1 = h.text
        try:
            end_idx = h1.find('명')
            h2 = int(h1[:end_idx])
            helpful.append(h2)
        except:
            helpful.append(0)
    return helpful

## 리뷰 가져오기

In [6]:
# 데이터를 저장할 데이터프레임 생성
df = pd.DataFrame({'customer_id': [], 'rating': [], 'title': [], 'country': [], 'date':[], 'content': [], 'helpful': []})

In [ ]:
# 각 별점마다 필터링 해서 리뷰를 가져온다.
for star in range(5):
    driver.find_element(By.XPATH, '//*[@id="cm_cr-view_opt_sort_filter"]/div[2]/div[2]/div[2]').click()
    time.sleep(1.5)
    
    driver.find_element(By.XPATH, f'/html/body/div[3]/div/div/ul/li[{star+2}]').click()
    time.sleep(1.5)
    
    print('-' * 20)
    print(f'{5-star}점 리뷰')
    print('-' * 20)
    
    # 리뷰 페이지 수
    # 여기서 오류 발생 시, 아마존 사이트의 언어 설정이 한국어로 되어 있는지 확인해주세요.
    count_text = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[4]')
    start_index = count_text.text.find('평가개,')
    end_index= count_text.text.find('리뷰개')
    
    count = int(count_text.text[start_index+4:end_index-1]) #  총 리뷰 개수
    print('총 리뷰 개수:', count)
    
    if count > 100:
        pages = 10
    elif count % 10 == 0:
        pages = count // 10 # 페이지 수
    else:
        pages = count // 10 + 1
    print('페이지 수:', pages)
    print('')
    time.sleep(1.5)
    
    for p in range(pages):

        customer_id = [] # 사용자 id
        rating = [] # 별점
        title = [] # 리뷰 제목
        country = [] # 리뷰가 작성된 국가
        date = [] # 리뷰 작성일
        content = [] # 리뷰 내용
        helpful = [] # 리뷰가 유용하다고 평가한 사람 수

        i = 2
        total = 12 # 페이지 당 10개의 리뷰를 가져온다.

        while i < total:
            
            try:
                a = driver.find_element(By.XPATH, f'/html/body/div[1]/div[2]/div/div[1]/div/div[1]/div[5]/div[3]/div/div[{i}]')
                time.sleep(1.5)
                
            except:
                break

            if a.text == '': # 미국이 아닌 다른 나라의 리뷰가 시작될 때는 공백 라인이 하나 들어가므로 패스한다.
                total = 13
                i += 1
                continue

            elif a.text == '이전\n다음': # 마지막 페이지의 마지막 리뷰에 도달했을 때는 반복문에서 나간다.
                break

            else:

                # 데이터 가져오기

                # customer_id
                cid = get_customer_id(i)
                customer_id.append(cid)
                time.sleep(1.5)

                # rating
                rt = get_rating(i)
                rating.append(rt)
                time.sleep(1.5)

                # title
                tt = get_title(i)
                title.append(tt)
                time.sleep(1.5)

                # country & date
                ct, dt = get_country_date(i)
                country.append(ct)
                date.append(dt)
                time.sleep(1.5)

                # content
                co = get_content(i)
                content.append(co)
                time.sleep(1.5)

                i += 1


        # helpful
        helpful = get_helpful()
        time.sleep(1.5)

        # 해당 페이지에서 가져온 데이터를 데이터프레임의 형태로 만들고, 이전에 가져온 데이터와 하나로 합쳐주기
        df_new = pd.DataFrame({'customer_id': customer_id, 'rating': rating, 'title': title, 'country': country, 'date': date,
                               'content': content, 'helpful': helpful})
        df = pd.concat([df, df_new], axis=0, ignore_index=True)

        # 다음 페이지로 이동 
        if (p != 0) & (p % 5 == 0):
            print(f'{p} pages done')
        
        try:
            next_page = driver.find_element(By.CLASS_NAME, 'a-last')
            next_page.click()
            time.sleep(1.5)
            
        except:
            break

--------------------
5점 리뷰
--------------------
총 리뷰 개수: 118
페이지 수: 10

5 pages done


In [ ]:
# 중복된 행이 있으면 제거
df.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
# 가져온 데이터 확인
df

In [ ]:
# csv 파일로 저장
brand = 'HP' # 여기에 브랜드 이름을 입력하세요.
product_name = 'OMEN i9-12900H' # 여기에 해당하는 제품을 입력하세요.
df.to_csv(f'{brand}_{product_name}_reviews.csv', index=False)

In [ ]:
# 저장된 파일이 잘 읽히는지 확인
pd.read_csv(f'{brand}_{product_name}_reviews.csv').head()